In [ ]:
from google.colab import files
uploaded = files.upload()

Saving customer_data_collection.csv to customer_data_collection (1).csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving product_recommendation_data.csv to product_recommendation_data (1).csv


In [ ]:
import sqlite3

# Connect to SQLite memory database (or use a file-based DB if needed)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create interactions table
cursor.execute('''
CREATE TABLE interactions (
    Customer_ID TEXT,
    Product_ID TEXT,
    Interaction_Type TEXT
)
''')

# Sample data insertion (simulating customer interactions)
sample_interactions = [
    ('C1000', 'P2000', 'purchase'),
    ('C1001', 'P2003', 'click'),
    ('C1002', 'P2002', 'add_to_cart'),
    ('C1003', 'P2004', 'purchase'),
    ('C1000', 'P2001', 'click'),
    ('C1001', 'P2002', 'purchase')
]

cursor.executemany("INSERT INTO interactions VALUES (?, ?, ?)", sample_interactions)
conn.commit()

# Function to get most frequently interacted products
def get_top_products():
    query = '''
    SELECT Product_ID, COUNT(*) as count FROM interactions
    GROUP BY Product_ID
    ORDER BY count DESC
    LIMIT 5
    '''
    top_products = pd.read_sql_query(query, conn)
    return top_products

# Get top products
top_products_df = get_top_products()
print("🎯 Top Recommended Products Based on Frequency:\n", top_products_df)

# Close database connection
conn.close()


🎯 Top Recommended Products Based on Frequency:
   Product_ID  count
0      P2002      2
1      P2004      1
2      P2003      1
3      P2001      1
4      P2000      1


In [ ]:
import numpy as np

class MultiArmedBandit:
    def __init__(self, product_ids):
        self.product_ids = product_ids
        self.product_rewards = {product: 0 for product in product_ids}
        self.product_counts = {product: 0 for product in product_ids}

    def select_product(self):
        """Use an epsilon-greedy approach to select a product."""
        epsilon = 0.1  # 10% chance of exploration
        if np.random.rand() < epsilon:
            return np.random.choice(self.product_ids)  # Explore
        else:
            return max(self.product_rewards, key=self.product_rewards.get)  # Exploit

    def update_rewards(self, product_id, reward):
        """Update rewards based on customer feedback."""
        self.product_counts[product_id] += 1
        self.product_rewards[product_id] += reward


In [ ]:
# Get top products from frequency-based system
top_product_ids = top_products_df["Product_ID"].tolist()

# Initialize Bandit Algorithm
bandit = MultiArmedBandit(top_product_ids)

# Simulate selecting a product for recommendation
selected_product = bandit.select_product()
print(f"🎯 Recommended Product using Bandit: {selected_product}")

# Simulate customer interaction (1 if they purchase, 0 if they ignore)
customer_feedback = np.random.choice([0, 1])
bandit.update_rewards(selected_product, customer_feedback)


🎯 Recommended Product using Bandit: P2002


In [ ]:
import sqlite3

# Connect to SQLite database (or create if not exists)
conn = sqlite3.connect("recommendation_memory.db")
cursor = conn.cursor()

# Create interactions table if not exists
cursor.execute('''
    CREATE TABLE IF NOT EXISTS interactions (
        customer_id TEXT,
        product_id TEXT,
        reward INTEGER,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
    )
''')
conn.commit()



In [ ]:
def store_interaction(customer_id, product_id, reward):
    """Stores the interaction into SQLite database."""
    cursor.execute(
        "INSERT INTO interactions (customer_id, product_id, reward) VALUES (?, ?, ?)",
        (customer_id, product_id, reward)
    )
    conn.commit()


In [ ]:
class MultiArmedBanditSQLite:
    def __init__(self, product_ids):
        self.product_ids = product_ids
        self.product_rewards = {product: 0 for product in product_ids}
        self.product_counts = {product: 0 for product in product_ids}

    def select_product(self):
        """Use an epsilon-greedy approach to select a product."""
        epsilon = 0.1
        if np.random.rand() < epsilon:
            return np.random.choice(self.product_ids)  # Explore
        else:
            return max(self.product_rewards, key=self.product_rewards.get)  # Exploit

    def update_rewards(self, customer_id, product_id, reward):
        """Update rewards in memory and store in database."""
        self.product_counts[product_id] += 1
        self.product_rewards[product_id] += reward

        # Store in SQLite
        store_interaction(customer_id, product_id, reward)


In [ ]:
# Load product IDs
top_product_ids = top_products_df["Product_ID"].tolist()

# Initialize Bandit with SQLite
bandit = MultiArmedBanditSQLite(top_product_ids)

# Simulate a recommendation for a customer
customer_id = "C1000"  # Example customer
selected_product = bandit.select_product()
print(f"🎯 Recommended Product for {customer_id}: {selected_product}")

# Simulate customer feedback (1 if they purchase, 0 if they ignore)
customer_feedback = np.random.choice([0, 1])
bandit.update_rewards(customer_id, selected_product, customer_feedback)

# Fetch interactions history from database
cursor.execute("SELECT * FROM interactions")
print(cursor.fetchall())  # View stored interactions


🎯 Recommended Product for C1000: P2001
[('C1000', 'P2000', 1, '2025-04-04 15:21:15'), ('C1000', 'P2001', b'\x01\x00\x00\x00\x00\x00\x00\x00', '2025-04-04 16:16:35')]


In [ ]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect("recommendation_memory.db")
cursor = conn.cursor()

# Drop the old table (only if needed)
cursor.execute("DROP TABLE IF EXISTS interactions")

# Commit changes
conn.commit()


In [ ]:
# Create interactions table with proper schema
cursor.execute('''
    CREATE TABLE IF NOT EXISTS interactions (
        customer_id TEXT,
        product_id TEXT,
        reward INTEGER,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
    )
''')

conn.commit()
print("✅ Table 'interactions' created successfully!")


✅ Table 'interactions' created successfully!


In [ ]:
cursor.execute("PRAGMA table_info(interactions)")
print(cursor.fetchall())  # This will show all columns in the table


[(0, 'customer_id', 'TEXT', 0, None, 0), (1, 'product_id', 'TEXT', 0, None, 0), (2, 'reward', 'INTEGER', 0, None, 0), (3, 'timestamp', 'DATETIME', 0, 'CURRENT_TIMESTAMP', 0)]


In [ ]:
[(0, 'customer_id', 'TEXT', 0, None, 0),
 (1, 'product_id', 'TEXT', 0, None, 0),
 (2, 'reward', 'INTEGER', 0, None, 0),
 (3, 'timestamp', 'DATETIME', 0, None, 0)]


[(0, 'customer_id', 'TEXT', 0, None, 0),
 (1, 'product_id', 'TEXT', 0, None, 0),
 (2, 'reward', 'INTEGER', 0, None, 0),
 (3, 'timestamp', 'DATETIME', 0, None, 0)]

In [ ]:
customer_id = "C1000"
selected_product = "P2000"
customer_feedback = 1  # Simulating a purchase

bandit.update_rewards(customer_id, selected_product, customer_feedback)

# Fetch interactions history from database
cursor.execute("SELECT * FROM interactions")
print(cursor.fetchall())  # This should return recorded interactions


[('C1000', 'P2000', 1, '2025-04-04 16:17:51')]


In [ ]:
pip install streamlit


In [ ]:
import streamlit as st
import pandas as pd
import sqlite3
import numpy as np
import time

# Load Data
@st.cache_data
def load_data():
    customer_df = pd.read_csv("customer_data_collection.csv")
    product_df = pd.read_csv("product_recommendation_data.csv")
    return customer_df, product_df

customer_df, product_df = load_data()

# Connect to SQLite
conn = sqlite3.connect("recommendation_memory.db", check_same_thread=False)
cursor = conn.cursor()

# Ensure the interactions table exists
cursor.execute('''
    CREATE TABLE IF NOT EXISTS interactions (
        customer_id TEXT,
        product_id TEXT,
        reward INTEGER,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
    )
''')
conn.commit()

# 💡 Multi-Armed Bandit Recommendation System
class BanditRecommendation:
    def __init__(self, product_df):
        self.products = product_df["Product_ID"].tolist()
        self.product_rewards = {p: 0 for p in self.products}

    def select_product(self):
        # Select a product based on highest reward probability
        return max(self.product_rewards, key=self.product_rewards.get)

    def update_rewards(self, customer_id, product_id, reward):
        # Update product rewards and store in DB
        self.product_rewards[product_id] += reward
        cursor.execute("INSERT INTO interactions (customer_id, product_id, reward) VALUES (?, ?, ?)",
                       (customer_id, product_id, reward))
        conn.commit()

# Initialize bandit model
bandit = BanditRecommendation(product_df)

# 🌟 Streamlit UI
st.title("📊 AI-Powered Product Recommendation System")

# Customer selection
customer_id = st.selectbox("Select Customer ID", customer_df["Customer_ID"].unique())

if st.button("Get Recommendation"):
    selected_product = bandit.select_product()
    st.success(f"🎯 Recommended Product: **{selected_product}**")

    # Simulating user interaction (purchase or ignore)
    feedback = st.radio("Did the customer purchase this product?", ["Yes", "No"])

    if feedback:
        reward = 1 if feedback == "Yes" else 0
        bandit.update_rewards(customer_id, selected_product, reward)
        st.write("✅ Feedback stored! The recommendation system is now learning in real-time.")

    # Show interaction history
    st.subheader("📌 Customer Interaction History")
    df_history = pd.read_sql("SELECT * FROM interactions WHERE customer_id=?", conn, params=(customer_id,))
    st.dataframe(df_history)

# Close connection
conn.close()


2025-04-04 16:18:55.359 No runtime found, using MemoryCacheStorageManager
2025-04-04 16:18:55.379 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:18:55.380 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:18:55.392 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:18:55.393 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:18:55.407 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:18:55.411 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:18:55.416 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:18:55.419 Thread 'MainThread':

In [ ]:
import streamlit as st
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt

# Database connection
def get_db_connection():
    conn = sqlite3.connect("recommendation_system.db")
    return conn

# Load customer and product data
@st.cache_data
def load_data():
    customer_df = pd.read_csv("customer_data_collection.csv")
    product_df = pd.read_csv("product_recommendation_data.csv")
    return customer_df, product_df

# Recommendation logic using frequency-based & Bandit approach
def recommend_product(customer_id, product_df):
    np.random.seed(42)  # For reproducibility
    recommended_product = product_df.sample(1).iloc[0]
    return recommended_product

# Store interaction in database
def store_interaction(customer_id, product_id, reward):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS interactions (
            customer_id TEXT,
            product_id TEXT,
            reward INTEGER
        )
    """)
    cursor.execute(
        "INSERT INTO interactions (customer_id, product_id, reward) VALUES (?, ?, ?)",
        (customer_id, product_id, reward),
    )
    conn.commit()
    conn.close()

# Streamlit UI
st.title("🔍 Hyper-Personalized Product Recommendation System")

customer_df, product_df = load_data()

# Select customer
customer_id = st.selectbox("Select Customer ID", customer_df["Customer_ID"].unique())

if st.button("Generate Recommendation"):
    recommended_product = recommend_product(customer_id, product_df)
    st.success(f"✅ Recommended Product: {recommended_product['Product_ID']} - {recommended_product['Category']}")

    # Simulate customer interaction (1 = purchased, 0 = ignored)
    reward = np.random.choice([0, 1])
    store_interaction(customer_id, recommended_product["Product_ID"], reward)

    st.write("🛒 **Recommendation Logged in Database**")

# Display Interaction History
if st.checkbox("Show Interaction History"):
    conn = get_db_connection()
    interaction_df = pd.read_sql_query("SELECT * FROM interactions", conn)
    conn.close()
    st.dataframe(interaction_df)

st.sidebar.title("📊 Insights")
if st.sidebar.button("Show Product Distribution"):
    fig, ax = plt.subplots()
    product_df["Category"].value_counts().plot(kind="bar", ax=ax)
    st.sidebar.pyplot(fig)

st.sidebar.write("📌 Developed by Teju 🚀")


2025-04-04 16:19:54.570 No runtime found, using MemoryCacheStorageManager
2025-04-04 16:19:54.586 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:19:54.594 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:19:54.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:19:54.617 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:19:54.620 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:19:54.629 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:19:54.643 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:19:54.652 Thread 'MainThread':

In [ ]:
import sqlite3

# Connect to database
conn = sqlite3.connect("recommendation_system.db")
cursor = conn.cursor()

# Check if 'interactions' table exists
cursor.execute("PRAGMA table_info(interactions)")
columns = cursor.fetchall()

# Print column names
print([col[1] for col in columns])


['customer_id', 'product_id', 'reward']


In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS interactions (
    customer_id TEXT,
    product_id TEXT,
    reward INTEGER
)''')
conn.commit()


In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
def fetch_interactions():
    conn = sqlite3.connect("recommendation_system.db")
    df = pd.read_sql("SELECT * FROM interactions", conn)
    conn.close()
    return df


In [ ]:
if st.button("View Interaction History"):
    df = fetch_interactions()
    st.write(df)


2025-04-04 16:21:40.964 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:21:40.966 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:21:40.967 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:21:40.967 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-04 16:21:40.968 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
import sqlite3
conn = sqlite3.connect("recommendation_system.db")
cursor = conn.cursor()
cursor.execute("SELECT * FROM interactions")
print(cursor.fetchall())
conn.close()


[]
